# loop

在编译原理课程项目中，进行自然循环识别的代码。

## 需要的库

- 正则表达式 re
- 序列化 json

In [1]:
import re
import json
import copy

## 读入json

In [25]:
filename = "../examples/vm_src03_blk.json"
vm_blk = {}
with open(filename, "r") as fp:
    vm_blk = json.load(fp)
blocks = vm_blk["blocks"]

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')

In [26]:
blocks

{'0': {'line_num': [0, 2],
  'next': [1, None],
  'code': ['I = M - 1', 'J = N', 'A = U1'],
  'defd': ['A', 'J', 'I'],
  'used': ['U1', 'M', 'N'],
  'in': ['U3', 'U1', 'U2', 'M', 'N'],
  'out': ['U3', 'U2', 'J', 'I']},
 '1': {'line_num': [3, 5],
  'next': [2, 3],
  'code': ['I = I + 1', 'J = J - 1', '? J < I : 7'],
  'defd': [],
  'used': ['J', 'I'],
  'in': ['U3', 'J', 'U2', 'I'],
  'out': ['U3', 'J', 'U2']},
 '2': {'line_num': [6, 6],
  'next': [3, None],
  'code': ['A = U2'],
  'defd': ['A'],
  'used': ['U2'],
  'in': ['U2', 'J', 'U3'],
  'out': ['U3', 'J', 'U2']},
 '3': {'line_num': [7, 8],
  'next': [4, 1],
  'code': ['I = U3', '? J > I : 3'],
  'defd': ['I'],
  'used': ['U3', 'J'],
  'in': ['U3', 'J', 'U2'],
  'out': ['U3', 'U2', 'J', 'I']},
 '4': {'line_num': [9, 9],
  'next': [None, None],
  'code': ['HALT'],
  'defd': [],
  'used': [],
  'in': [],
  'out': []}}

# 给基本块加入前驱节点属性

In [27]:
for n,b in blocks.items():
    blocks[n]["pre"] = set()

for n,b in blocks.items():
    for i in b["next"]:
        if i!=None:
            blocks[str(i)]["pre"] = blocks[str(i)]["pre"] | {n}

for n,b in blocks.items():
    # blocks[n]["pre"] = list(blocks[n]["pre"])
    print(n,b["pre"],b["next"])

0 set() [1, None]
1 {'3', '0'} [2, 3]
2 {'1'} [3, None]
3 {'1', '2'} [4, 1]
4 {'3'} [None, None]


# 构建支配关系

In [30]:
for n,b in blocks.items():
    blocks[n]["dom"] = set()

for n,b in blocks.items():
    blocks[n]["dom"] = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
blocks[str(0)]["dom"] = {'0'}

for n,b in blocks.items():
    print(n,b["dom"])

flag = True
i = 0
while flag:
    flag = False
    i+=1
    print(i)
    for n,b in blocks.items():
        if n!='0':
            domset_ori = copy.deepcopy(blocks[n]["dom"])
            domset = copy.deepcopy(blocks[n]["dom"])
            for j in blocks[n]["pre"]:
                domset = domset | blocks[j]["dom"]
            domset = {n} | domset
            if (domset!=domset_ori):
                flag = True
            blocks[n]["dom"] = domset
    for n,b in blocks.items():
        print(n,b["dom"])

0 {'0'}
1 {'3', '0', '1', '2', '4'}
2 {'3', '0', '1', '2', '4'}
3 {'3', '0', '1', '2', '4'}
4 {'3', '0', '1', '2', '4'}
1
0 {'0'}
1 {'1', '3', '2', '4', '0'}
2 {'1', '3', '2', '4', '0'}
3 {'1', '3', '2', '4', '0'}
4 {'1', '3', '2', '4', '0'}
